In [1]:
import numpy as np
from sklearn import preprocessing
#inputları standardize etmek için sklearn imkanları kullanılacak
#neredeyse her zaman tüm inputlar standardize edilir. Bu sayede algoritmanın kalitesi artar.

#csv dosyası yüklenir. delimiter =',' olayı dosya içeriğini virgüllü yerleri böl demek. split gibi.
raw_csv_data=np.loadtxt('Audiobooks_data.csv',delimiter= ',')

#bizim inputumuz csv deki ilk ve son sütun hariç hepsi. Bu yüzden ilk ve son sütunu almıyoruz.
unscaled_inputs_all=raw_csv_data[:,1:-1]#1 den -1 e kadar al dedik
targets_all=raw_csv_data[:,-1] #son sütun bizim targetımız. -1 i al demiş olduk

### Balance the dataset
1. Önce target içindeki 1 leri sayacağız
2. Sonra 1 kadar 0 olanları alacağız. diğer 0 ları sileceğiz

Eğer 2 den fazla sınıf için bu programı kullanacaksak ona göre burayı balance ederiz

In [2]:
#Tüm target sütununu toplarsak kaç tane 1 olduğunu buluruz
num_one_targets=int(np.sum(targets_all))
#şimdi 1 kadar 0 alıyoruz
zero_targets_counter=0 #0ları sayacak bir counter tanımladık.
indices_to_remove=[] #slilinecek indexler için değişken tanımladık.

for i in range(targets_all.shape[0]): #targets_all bir numpy.darray dir.Bu yüzden targets_all un shape i (0 sütununda), vektörün uzunluğunu verir. Yni bize tüm target sayısını gösterir.
    if targets_all[i]==0:
        zero_targets_counter+=1
        if zero_targets_counter>num_one_targets:
            indices_to_remove.append(i) #1lerin sayısından fazla olduğu anda o indexi not aldık ki gerisini silelim.
#yani döngüde eğer sıfırsa counterı 1 artır ve eğer counter>1 sayısındansa o indexi not al. Bu indexler zaten 0 olduğundan sileceğiz.

unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
#np.delete(array, obj to delete, axis) silmeye yarar. Burada indicies_to_temove dizisindeki indeksleri tüm input dizisinden siliyoruz yani inputlarda o indexlerin tüm bilgilerinin silinmiş halini yeni diziye atıyoruz.
#bu fonksiyonda unscaled inputs all dan indices_to_remove dizisindeki elemanlara denk gelen satırları si dedik.axis 1 olsaydı index dizisindeki elemanlara denk gelen sütunları silerdi.
targets_equal_priors=np.delete(targets_all, indices_to_remove,axis=0)

#input ve target balance edildi.

### Standardize the  (normalization)
İnputların ağırlıkları farklı olduğundan büyük olan daha ağır küçük olanın daha hafif olması için standardization yapılır.inputlar şu anda unscaled bunları standardize edeceğiz.(x-m)/stddev

In [3]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors) #değişkendeki tüm elemanları standardize eder.

### Shuffle the data
inputlarıda targetlarıda random karıştırıyoruz. Datayı batch etmeden önce mutlaka karıştırıyoruz. Çünkü farklı tarihlerde farklı satıs miktarı olmuş olabilir.

In [4]:
shuffled_indices=np.arange(scaled_inputs.shape[0]) #np.arrange([start],stop) verilen aralıkta eşit aralı elemanlardan oluşan dizi döndürür.
#arrange fonksiyonunda input sayısı kadar index ürettik.(mesela 10 tane input varsa 1 den 10 a kar değerler olan bir dizi oluşturduk)
np.random.shuffle(shuffled_indices)#sonra bu indexleri karıştırdık.

shuffled_inputs=scaled_inputs[shuffled_indices]#karıştırılan indexli inputlar yeni değişkene verildi.bu işlemde sadece satırlar aşağı yukarı yönlü karıştırılmış oldu
shuffled_targets=targets_equal_priors[shuffled_indices]#aynı şekilde target da

### Split the data set val, test, train
setleri 80-10-10 şeklinde böleceğiz.

In [5]:
samples_count=shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count
#kayıp 1 varsa onu da hesaba katmış olduk

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
#80 den 80+10 a kadar dedik
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets=shuffled_targets[train_samples_count + validation_samples_count:]

#tüm data setini balance ettik ama bunları ayrı ayrı balance etmedik.
#tüm data setleri için şunları yazdırıyoruz

print(np.sum(train_targets),train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets),validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets),test_samples_count,np.sum(test_targets)/test_samples_count)
#hepsinin yaklaşık %50 olması gerek. yani taerget setlerdeki 0 ve 1 sayısı hemen hemen eşit

1790.0 3579 0.500139703828
217.0 447 0.485458612975
230.0 448 0.513392857143


### Save the datasets in .npz file

In [6]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez("Audiobooks_data_validation", inputs=validation_inputs, targets=validation_targets)
np.savez("Audiobooks_data_test", inputs=test_inputs, targets=test_targets)

############# artık preprocess işlemi tamam ###########

Bu programda veriyi hazırlayıp setlere böldük. Preprocessing işlemi tamamlandı. Data setleri ayrı npz dosyalarına attık ve bizim işimiz onlarla olduğuna göre artık başka bir sayfayla devam edebiliriz. 